In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math


In [4]:
weather_los_angeles = pd.read_csv('../resources/dataset_weather/weather_los_angeles.csv')

In [3]:
def calculate_heat_index(temp, humidity):
    # Constants for the formula
    c1 = -42.379
    c2 = 2.04901523
    c3 = 10.14333127
    c4 = -0.22475541
    c5 = -0.00683783
    c6 = -0.05481717
    c7 = 0.00122874
    c8 = 0.00085282
    c9 = -0.00000199

    if temp >= 27:
        hi = (c1 + (c2 * temp) + (c3 * humidity) + (c4 * temp * humidity) +
              (c5 * temp ** 2) + (c6 * humidity ** 2) +
              (c7 * temp ** 2 * humidity) + (c8 * temp * humidity ** 2) +
              (c9 * temp ** 2 * humidity ** 2))
        return hi
    else:
        return temp

def calculate_wind_chill(temp, wind_speed):
    if temp <= 10 and wind_speed >= 4.8:
        wci = 13.12 + 0.6215 * temp - 11.37 * (wind_speed ** 0.16) + 0.3965 * temp * (wind_speed ** 0.16)
        return wci
    else:
        return temp

def calculate_humidex(temp, dew_point):
    e = 6.112 * math.exp((17.67 * (dew_point/100)) / ((dew_point/100) + 243.5))
    h = temp + 0.5555 * (e - 10.0)
    return h

def normalize_heat_index(hi):
    return (hi - 70) / (130 - 70) * 100

def normalize_wind_chill(wci):
    return (wci + 50) / (50 + 50) * 100

def normalize_humidex(h):
    return (h - 20) / (50 - 20) * 100

def calculate_cwcs(temp, humidity, wind_speed, dew_point):
    # Calculate indices
    hi = calculate_heat_index(temp, humidity)
    wci = calculate_wind_chill(temp, wind_speed)
    humidex = calculate_humidex(temp, dew_point)

    # Normalize the indices
    normalized_hi = normalize_heat_index(hi)
    normalized_wci = normalize_wind_chill(wci)
    normalized_humidex = normalize_humidex(humidex)

    # Determine weights based on temperature
    if temp > 25:
        weight_hi = 0.5
        weight_wci = 0.1
        weight_humidex = 0.4
    elif temp < 10:
        weight_hi = 0.1
        weight_wci = 0.8
        weight_humidex = 0.1
    else:
        weight_hi = 0.33
        weight_wci = 0.34
        weight_humidex = 0.33

    # Calculate CWCS
    cwcs = (weight_hi * normalized_hi) + (weight_wci * normalized_wci) + (weight_humidex * normalized_humidex)
    return cwcs

In [ ]:
weather_los_angeles['weather_score'] = calculate_cwcs()